# Groth16

At Eurocrypt 2016, Jens Groth published a [paper](https://eprint.iacr.org/2016/260.pdf) where he formalized a proving system that significantly improved performance compared to Pinocchio. In particular, for arithmetic circuits, proofs consist of only 2 elements in G1 and 1 in G2, compared to Pinocchio’s 7 elements in G1, and 1 in G2. Furthermore, verification is faster, with just 3 pairings instead of 12. The CRS is also shorter.

Groth16 needs one setup for each circuit compared to universal setup for PLONK protocol

## Problem Statement

Use Groth16 to prove a statement without leaking any other information.

Statement:

$x^3 + x + 5 = 35$

solution $x = 3$

this value will remain private, it will never be sent to the verifier.


## Arithmetic Circuit

The equation $x^3 + x + 5 = 35$ can be break down into an arithmetic circuit:

```mermaid
%%{ init : { "theme" : "default", "flowchart" : { "curve" : "linear" }}}%%
graph LR
A(x)
B(x)
M1($$*$$)

A-->M1
B-->M1
M1-->sym1

C(x)
M2($$*$$)
sym1-->M2
C-->M2
M2-->y

M3($$+$$)
D(x)-->M3
y-->M3
M3-->sym2

M4($$+$$)
5-->M4
sym2-->M4
M4-->Cut(out=35)
```

From this arithmetic circuit, we can define a set of constraints which has the form $O = L * R$

$
\begin{aligned}
sym_1 &= x * x \\
y &= sym_1 * x \\
sym_2 &= x + y \quad &\rarr sym_2 = (x + y) * 1 \\
out &= sym_2 + 5 &\rarr out = (sym_2 + 5) * 1 \\
\end{aligned}
$


## Rank 1 Constraint System

We have a proper $O = L * R$ form, we can create 4 matrices.
The first matrix is named $w$. It is known as the *witness* matrix.
The value of $w$ elements are known by the prover.

$\vec{s} = (1, out, x, y, sym_1, sym_2)$

Then, we can produce 3 matrices that will represent the constraints.
It sort of represents the way the $w$ elements are interconnected.
Those 3 matrices will be named $A$, $B$ and $C$.

$
\begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & \fbox{1} & \fbox{1} & 0 & 0 \\
5 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
1 \\
out \\
\fbox{x} \\
\fbox{y} \\
sym_1 \\
sym_2 \\
\end{bmatrix}
\odot
\begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
\fbox{1} & 0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}
\begin{bmatrix}
\fbox{1} \\
out \\
x \\
y \\
sym_1 \\
sym_2
\end{bmatrix}
\text {=}
\begin{bmatrix}
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & \fbox{1} \\
0 & 1 & 0 & 0 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
1 \\
out \\
x \\
y \\
sym_1 \\
\boxed{sym_{2}} \\
\end{bmatrix} \\
$

The final R1CS representation is $(A \cdot w)\odot(B \cdot w) = C \cdot w$
Each column of the matrices $A,B,C$ corresponds to an element of $w$.
Each row of the matrices $A,B,C$ corresponds to one of the constraints of the arithmetic circuit.

For example, let's take the 3rd row of $L, R, O$.
In $L$, the value $1$ is set at the $3^{rd}$ and $4^{th}$ column.
The $3^{rd}$ and $4^{th}$ column of $w$ is the $x, y$ variable separately.
So, the 3rd row of $A$ encodes $1 \cdot x + 1 \cdot y$.
The same logic applies to $B$, its 3rd row encodes $1 \cdot 1$.
And for $C$, the 3rd row encodes $1 \cdot sym_2$.
If we put it all together, the $Cw = Aw \odot Bw$ gives $sym_2 = (x + y) * 1$ which correspond to our 3rd constraints defined previously.

Let's verify that the computing of $Os = Ls \odot Rs$ is correct.
We will use Python and numpy for this.
As we are working on a finite field, negative number will be modified.
For example, in $F_p$, $-1 (mod\ p)= p-1$.

In [1]:
import galois
import numpy as np
import py_ecc.bn128 as bn128
import time

print("curve order: ", bn128.curve_order)
print("field modulus: ", bn128.field_modulus)

# p = 101
p = bn128.curve_order

start_time = time.time()
GF = galois.GF(p)
print(f"GF{p} initializing takes {time.time() - start_time} seconds")

x = 3
sym_1 = x * x
y = sym_1 * x
sym_2 = y + x
out = sym_2 + 5

w = GF([1, out, x, y, sym_1, sym_2])
print("w = ", w)

A = GF(
    [
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [5, 0, 0, 0, 0, 1],
    ]
)

B = GF(
    [
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
    ]
)

C = GF(
    [
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0],
    ]
)

Aw = np.dot(A, w)
Bw = np.dot(B, w)
Cw = np.dot(C, w)

print("Aw = ", Aw)
print("Bw = ", Bw)


AwBw = np.multiply(Aw, Bw)  # Aw * Bw
print("Aw * Bw = ", AwBw)
print("Cw = ", Cw)

assert np.all([AwBw, Cw]), "Az o Bz != Cz"

/Users/cameo.gao/zksnark/py-snark/.venv/lib/python3.12/site-packages/eth_utils/network.py:61: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  networks = initialize_network_objects()
/Users/cameo.gao/zksnark/py-snark/.venv/lib/python3.12/site-packages/eth_utils/network.py:61: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  networks = initialize_network_objects()


curve order:  21888242871839275222246405745257275088548364400416034343698204186575808495617
field modulus:  21888242871839275222246405745257275088696311157297823662689037894645226208583
GF21888242871839275222246405745257275088548364400416034343698204186575808495617 initializing takes 126.77771019935608 seconds
w =  [ 1 35  3 27  9 30]
Aw =  [ 3  9 30 35]
Bw =  [3 3 1 1]
Aw * Bw =  [ 9 27 30 35]
Cw =  [ 9 27 30 35]


## QAP

The next step is to transform this R1CS into a Quadratic Arithmetic Program (QAP).

This transformation consists of turning our constraint matrices into polynomials.
The main goal of this transformation is **succinctness**.
Thanks to the Schwartz-Zippel Lemma, we know that evaluating two polynomials at a random point $x$ allows to nearly be certain that they are the same polynomial if they return the same value $y$.
In summary, if $P_1(\tau) == P_2(\tau)$, then $P1 == P2$, with $\tau$ being a random number.
This evaluation is way more succinct than matrices evaluation.

To achieve succinctness, we will derivate a QAP from our R1CS.
Our goal is to get $A(x) * B(x) = C(x)$ if $Aw \odot Bw = Cw$.
The transformation is made possible thanks to a ring homomorphism between matrices and polynomials.

The created polynomials must encode the same information as our R1CS matrices, but uniquely at the points at which it will be evaluated.
As we have 4 constraints, we will evaluate our polynomials at $x=[1,2,3,4]$, or at $x = \{1, \omega, \omega^2, \ldots, \omega^{n-1} \}$ to speed up the computation for verifier.

### Root of Unity

The domain that is used for polynomial interpolation consists of roots of unity. The $n^{th}$ roots of unity of a field are the field elements $\omega$ that satisfy $\omega^n =1$.

For example, the base field $\mathbb F_{17}$ has exactly $4^{th}$ roots of unity. Finding the $4^{th}$ roots of unity in $\mathbb F_{17}$ isn't too difficult:

Suppose $\omega_0, ..., \omega_{n-1}$’s are the roots of unity such that

- roots of $x^n = 1$
- $\omega_i = \omega_1^i = \omega^i$
- $\omega_i \neq \omega_j$


### R1CS to QAP
For each column of each matrix, we will interpolate a polynomial.
The interpolation will be done using Lagrange interpolation.
The interpolated polynomial will encode all the values of the column.

For example, the third column of $A$ is:
$$\begin{bmatrix} 1 \\ 0 \\ 1 \\ 0 \\ \end{bmatrix}$$

So, the interpolated polynomial $A_3(x)$ must return the following values:

$
\begin{aligned}
A_3(1) &= 1 \\
A_3(\omega) &= 0 \\
A_3(\omega^{2}) &= 1 \\
A_3(\omega^{3}) &= 0 \\
\end{aligned}
$

$
\left(\sum_{i=0}^n u_i \cdot l_i(x)\right)  \cdot \left(\sum_{i=0}^n u_i \cdot r_i(x)\right) - \left(\sum_{i=0}^n u_i \cdot o_i(x)\right) = t(x)h(x)
$

$
\overrightarrow{A(X)}=(A_0(X),A_1(X),A_2(X),A_3(X),A_4(X),A_5(X)) \\
\overrightarrow{B(X)}=(B_0(X),B_1(X),B_2(X),B_3(X),B_4(X),B_5(X)) \\
\overrightarrow{C(X)}=(C_0(X),C_1(X),C_2(X),C_3(X),C_4(X),C_5(X))
$

$
A(X)=\langle \overrightarrow{A(X)},w \rangle \\
B(X)=\langle \overrightarrow{B(X)},w \rangle \\
C(X)=\langle \overrightarrow{C(X)},w \rangle
$


In [2]:
# import sympy as sp

# number of rows
m = A.shape[0]

# number of columns
n = A.shape[1]

omega = GF.primitive_root_of_unity(m)

root_of_unities = GF([omega**i for i in range(m)])
display(root_of_unities)

# get smallest root, False in the parameter
# root_of_unities_1 = sp.nthroot_mod(1, m, p, True)

# column 3 of matrix A
A3 = A.T[2]
display(A3)


# lagrange interpolate the column to a polynomial
A3_poly = galois.lagrange_poly(root_of_unities, A3)
display(A3_poly)

# # Checking each column element
print(f"A3_poly({omega**0}) == 1: {A3_poly(omega**0) == 1}")
print(f"A3_poly({omega**1}) == 0: {A3_poly(omega**1) == 0}")
print(f"A3_poly({omega**2}) == 1: {A3_poly(omega**2) == 1}")
print(f"A3_poly({omega**3}) == 0: {A3_poly(omega**3) == 0}")

GF([                                                                            1,
    21888242871839275217838484774961031246007050428528088939761107053157389710902,
    21888242871839275222246405745257275088548364400416034343698204186575808495616,
                       4407920970296243842541313971887945403937097133418418784715],
   order=21888242871839275222246405745257275088548364400416034343698204186575808495617)

GF([1, 0, 1, 0],
   order=21888242871839275222246405745257275088548364400416034343698204186575808495617)

Poly(10944121435919637611123202872628637544274182200208017171849102093287904247809x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247809, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))

A3_poly(1) == 1: True
A3_poly(21888242871839275217838484774961031246007050428528088939761107053157389710902) == 0: True
A3_poly(21888242871839275222246405745257275088548364400416034343698204186575808495616) == 1: True
A3_poly(4407920970296243842541313971887945403937097133418418784715) == 0: True


In [3]:
# number of rows
m = A.shape[0]

# number of columns
n = A.shape[1]

print(f"rows = {m}, columns = {n}")

matrixes = [A, B, C]

omega = GF.primitive_root_of_unity(m)
assert omega ** (m) == 1, f"omega {omega} is not a root of unity"
print("omega = ", omega)

root_of_unities = GF([omega**i for i in range(m)])

constraint_poly_vectors = []
constraint_witness_poly_vectors = []

for matrix in matrixes:
    poly_vector = []

    for values in matrix.T:
        assert len(values) == len(root_of_unities), "interpolate pair should be matched"

        # lagrange interpolate the column to a polynomial
        poly = galois.lagrange_poly(root_of_unities, values)
        poly_vector.append(poly)

    constraint_poly_vectors.append(poly_vector)
    display(poly_vector)

for poly_vector in constraint_poly_vectors:
    constraint_witness_poly = galois.Poly([0], field=GF)
    for poly_i, witness_i in zip(poly_vector, w):
        constraint_witness_poly += poly_i * witness_i
    constraint_witness_poly_vectors.append(constraint_witness_poly)

display(constraint_witness_poly_vectors)

A_witness_poly = constraint_witness_poly_vectors[0]
B_witness_poly = constraint_witness_poly_vectors[1]
C_witness_poly = constraint_witness_poly_vectors[2]

print(f"poly (A(x)*w) = {A_witness_poly * B_witness_poly}")
print(f"poly (C(x)*w) = {C_witness_poly}")

r = GF.Random()

output = """
random r = {0}，
poly_a(r) = {1},
poly_b(r) = {2},
poly_a(r) * poly_b(r) = {3},
poly_c(r) = {4}
"""
formatted_output = output.format(
    r,
    A_witness_poly(r),
    B_witness_poly(r),
    A_witness_poly(r) * B_witness_poly(r),
    C_witness_poly(r),
)
print(formatted_output)

rows = 4, columns = 6
omega =  21888242871839275217838484774961031246007050428528088939761107053157389710902


[Poly(5472060717959818811071502649184623575313733564963940340845922463416975604798x^3 + 5472060717959818805561601436314318772137091100104008585924551046643952123903x^2 + 16416182153879456411174903096072651513234630835452094002852281723158832890819x + 16416182153879456416684804308942956316411273300312025757773653139931856371714, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(10944121435919637611123202872628637544274182200208017171849102093287904247809x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247809, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(5472060717959818805561601436314318772137091100104008585924551046643952123904x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371713x^2 + 5472060717959818805561601436314318772137091100104008585924551046643952

[Poly(10944121435919637612225183115202698504909510693180003522833376376642508943987x^3 + 21888242871839275221144425502683214127913035907444047992713929903221203799438x + 10944121435919637611123202872628637544274182200208017171849102093287904247809, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(10944121435919637610021222630054576583638853707236030820864827809933299551630x^3 + 1101980242574060960635328492971986350984274283354604696179x + 10944121435919637611123202872628637544274182200208017171849102093287904247809, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204

[Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(5472060717959818806663581678888379732772419593075994936908825329998556820083x^3 + 5472060717959818805561601436314318772137091100104008585924551046643952123904x^2 + 16416182153879456415582824066368895355775944807340039406789378856577251675534x + 16416182153879456416684804308942956316411273300312025757773653139931856371713, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(16416182153879456415582824066368895355775944807340039406789378856577251675534x^3 + 5472060717959818805561601436314318772137091100104008585924551046643952123904x^2 + 5472060717959818806663581678888379732772419593075994936908825329998556820083x + 16416182153879456416684804308942956316411273300312025757773653139931856371713, GF(2188824287183927522224640574525727508854836440041603434369820418657

[Poly(5472060717959818834213087743239903748655631917375653711515682413863674224545x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371710x^2 + 5472060717959818776910115129388733795618550282832363460333419679424230023250x + 16416182153879456416684804308942956316411273300312025757773653139931856371732, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(21888242871839275220042445260109153167277707414472061641729655619866599103260x^3 + 2203960485148121921270656985943972701968548566709209392358x + 2, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)),
 Poly(5472060717959818814377443376906806457219719043879899393798745313480789693329x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371707x^2 + 5472060717959818796745759495721831087054463156328117778050356779807114554469x + 16416182153879456416684804308942956316411273300312025757773653139931856371738, GF(218882428718392

poly (A(x)*w) = 5472060717959818834764077864526934228973296163861646887007819555540976572641x^6 + 5472060717959818811622492770471654055631397811449933516338059605094277952886x^5 + 5472060717959818805561601436314318772137091100104008585924551046643952123891x^4 + 5472060717959818814377443376906806457219719043879899393798745313480789693329x^3 + 10944121435919637581920726444416022087437977136450378870765833584390879799066x^2 + 21888242871839275207369672470507452119971429745294218605410501361288645097200x + 10944121435919637611123202872628637544274182200208017171849102093287904247847
poly (C(x)*w) = 5472060717959818814377443376906806457219719043879899393798745313480789693329x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371707x^2 + 5472060717959818796745759495721831087054463156328117778050356779807114554469x + 16416182153879456416684804308942956316411273300312025757773653139931856371738

random r = 616760593505125227893305455793871900586131874122516570761060282

### QAP balancing - $t(x)$ part


Our QAP $\sum\limits_{i=0}^{m} w_i a_i(x) \sum\limits_{i=0}^{m} w_i b_i(x) = \sum\limits_{i=0}^{m} w_i c_i(x)$ has an issue.
The left term will have a degree greater than the right term.
Even if this does not have any impact on the points at which we evaluate those polynomials, we will fix it by adding a *balancing term*.

The balancing term is a set of two polynomial $h(x)$ and $Z(x)$ that will be added to the right term:
$\sum\limits_{i=0}^{m} w_i a_i(x) \sum\limits_{i=0}^{m} w_i b_i(x) = \sum\limits_{i=0}^{m} w_i c_i(x) + h(x)Z(x)$.

$
\begin{aligned}
Degree(A(x)) &= m - 1 \\
Degree(B(x)) &= m - 1 \\
Degree(C(x)) &= m - 1 \\
Degree(Z(x)) &= m \\
Degree(H(x)) &= 2(m - 1) - m = m - 2 \\
\end{aligned}
$


We need this balancing term to be zero at the points we evaluate the polynomials. To do so, we use $t(x)$ as a sort of nullifier.
As we evaluate at $x=[1,\omega, \ldots, \omega^{m-1}]$, we can easily find that $Z(x)= (x-1)(x-\omega)\ldots(x-\omega^{m-1})$. Remember that as we are in a finite field, negative values are handled differently.

Then, we need to compute $h(x)$:

$A(X) \cdot B(X) - C(X) = 0 \; \forall \; X \in \{1, \omega, \omega^2, \cdots, \omega^{n-1}\}$

The composite polynomial can be constructed from:

$
f(x) = A(x) \cdot B(x) - C(x) = 0, \; \forall \; x \in \{1, \omega, \omega^2, \cdots, \omega^{n-1}\} \\
Z(x)=(x−1)(x−\omega) \cdots (x−\omega^{n-1}) = x^n - 1 \\
f(x) = Z(x) \cdot H(x), \; \forall \; x \in \mathbb{F} \\
A(x) \cdot B(x) - C(x) = t(x) \cdot H(x), \; \forall \; x \in \mathbb{F} \\
$

In [4]:
poly_z = galois.Poly.Degrees([m, 0], coeffs=([1, -1]), field=GF)

poly_qap = A_witness_poly * B_witness_poly - C_witness_poly

print(f"poly_qap = {poly_qap}")
print(f"poly_z = {poly_z}, degree(z) = {poly_z.degree}")

for i in range(0, m):
    x = np.power(omega, i)
    assert poly_qap(x) == 0
    assert poly_z(x) == 0


poly_h = poly_qap // poly_z

rem_poly = poly_qap % poly_z
assert rem_poly == 0

print(f"poly_h = {poly_h}")

# r must be in the same finite field
# GF2 = galois.GF(65537)
# r = GF2.Random()

r = GF.Random()
assert poly_qap(r) == poly_z(r) * poly_h(r)

poly_qap = 5472060717959818834764077864526934228973296163861646887007819555540976572641x^6 + 5472060717959818811622492770471654055631397811449933516338059605094277952886x^5 + 5472060717959818805561601436314318772137091100104008585924551046643952123891x^4 + 16416182153879456387482327880730340859575068236554387456690384631034831922976x^2 + 16416182153879456410623912974785621032916966588966100827360144581481530542731x + 16416182153879456416684804308942956316411273300312025757773653139931856371726
poly_z = x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495616, degree(z) = 4
poly_h = 5472060717959818834764077864526934228973296163861646887007819555540976572641x^2 + 5472060717959818811622492770471654055631397811449933516338059605094277952886x + 5472060717959818805561601436314318772137091100104008585924551046643952123891


## Trusted Setup

### Vanilla Check

$A(x) \cdot B(x) - C(x) = t(x) \cdot H(x), \; \forall \; x \in \mathbb{F}$


In [5]:
import py_ecc.bn128.bn128_pairing as pairing


def commit_poly(poly: galois.Poly, trusted_points, verbose=False):
    coeffs = poly.coefficients()[::-1]

    assert len(coeffs) <= len(trusted_points)

    if verbose:
        print("trusted points = ")
        display(trusted_points)

    terms = [
        bn128.multiply(trusted_points[i], int(coeff)) for i, coeff in enumerate(coeffs)
    ]

    evaluation = terms[0]
    for term in terms[1:]:
        evaluation = bn128.add(evaluation, term)

    if verbose:
        print(f"{evaluation}")
    return evaluation


# y^2 = x^3 + 3
# p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

print("trusted setup vanilla check")
print("-" * 30)
print("toxic waste for tau commitment")
# start PHASE I setup
tau = GF.Random()
# G1[τ^0], G1[τ^1], ..., G1[τ^{m}]]
tau_G1 = [bn128.multiply(bn128.G1, int(tau**i)) for i in range(poly_qap.degree + 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^{m}]
tau_G2 = [bn128.multiply(bn128.G2, int(tau**i)) for i in range(poly_qap.degree + 1)]

commitment_A_G1 = commit_poly(A_witness_poly, tau_G1)
commitment_B_G2 = commit_poly(B_witness_poly, tau_G2)
commitment_C_G1 = commit_poly(C_witness_poly, tau_G1)

# h(x)*z(x)
# [τ^0*Z(τ)]G1, [τ^1*Z(τ)]G1, ... [τ^{m-2}*Z(τ)]G1
HZ_G1 = [
    bn128.multiply(bn128.G1, int(tau**i * poly_z(tau)))
    for i in range(poly_h.degree + 1)
]

commitment_HZ_G1 = commit_poly(poly_h, HZ_G1)
commitment_C_G1 = bn128.add(commitment_C_G1, commitment_HZ_G1)

print("vanilla check e(A1, B2) == e(C1, G2)")
assert pairing.pairing(commitment_B_G2, commitment_A_G1) == pairing.pairing(
    bn128.G2, commitment_C_G1
)

print("-" * 30)

trusted setup vanilla check
------------------------------
toxic waste for tau commitment
vanilla check e(A1, B2) == e(C1, G2)
------------------------------


## Trusted Setup

### Prover Soundness (with $\alpha$ $\beta$ security features)

When the verifier receives points A, B, and C, they cannot determine if these points are genuine results of an evaluation or merely a trick by the prover. For this very reason, we need to introduce two additional parameters, $\alpha$ and $\beta$ as random shift,$\alpha$ and $\beta$ EC points are used to avoid an attacker to be able to forge valid proofs for invalid statements. These parameters must be provided by a trusted setup agent. 

Groth 16 requires sampling three random field elements to generate the proving and verifying key: $\alpha, \beta, \tau$. These are part of **toxic waste** and should be discarded and wholly forgotten once the keys have been generated.

Public SRS:
- powers of $\tau$ for $A$:
  - $\{ \tau^i[G_1]_1 \}_{i=0}^{i=n-1}$
- random shift for $A$:
  - [\alpha G_1]_1
- powers of $\tau$ for $B$:
  - $\{ \tau^i[G_2]_2 \}_{i=0}^{i=n-1}$
- random shift for $B$:
  - $[\beta G_1]_1$
- powers of $\tau$ for $C$
  - $\{(C_{i} (\tau) + \beta A_i (\tau) + \alpha B_i (\tau))[G_1]_1\}$
- powers of $\tau$ for $h(\tau)Z_D(\tau)$ (for prover):
  - $Z_k (\tau) = \{\tau^k Z_D(\tau)[G_1]_1\} , \forall \ k = \{0, 1, 2, \ldots, m - 2 \}$.

For Verifier(public input):
$K_i^v(\tau) = (C_{i} (\tau) + \beta A_i (\tau) + \alpha B_i (\tau)) g_1, \forall \ i = \{0, 1, 2, \ldots, k \}$.

For Prover(witness)
$K_i^p(\tau) = (C_{i} (\tau) + \beta A_i (\tau) + \alpha B_i (\tau)) g_1,
\forall \ i = \{k+1, k+2, \ldots, n \}$


The proving key consists of the following elements:

1.  $[\alpha G_1]_1$
2.  $[\beta G_1]_1$
3.  $[\beta G_2]_2$
4.  $[\delta G_1]_1$
5.  $[\delta G_2]_2$
6.  $[A_0 (\tau)]_1, [A_1(\tau)]_1 , \ldots , [A_n(\tau)]_1$
7.  $[B_0 (\tau)]_1, [B_1(\tau)]_1 , \ldots , [B_n(\tau)]_1$
8.  $[B_0 (\tau)]_2, [B_1(\tau)]_2 , \ldots , [B_n(\tau)]_2$
9.  $[K_{ k + 1 }^p (\tau)] , [ K_{ k + 2 }^p (\tau)] , \ldots , [K_n^p (\tau)]$
10. $[Z_0(\tau)] , [Z_1(\tau)] , \ldots , [ Z_{ m - 1 } (\tau)]$

The verifying key is much shorter and will contain in addition the value of one pairing because that value is constant:

1.  $[\alpha]_1 \dagger [\beta]_2$
2.  $[\gamma]_2$
3.  $[\delta]_2$
4.  $[K_{0}^{v}(\tau)]_1, [K_{1}^{v}(\tau)]_1, \ldots, [K_{k}^{v}(\tau)]_1$

In [6]:
# y^2 = x^3 + 3
# p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

print("trusted setup phase one")
print("-" * 30)
print("toxic waste for tau commitment")
# start PHASE I setup
tau = GF.Random()
# G1[τ^0], G1[τ^1], ..., G1[τ^{m}]]
tau_G1 = [bn128.multiply(bn128.G1, int(tau**i)) for i in range(poly_qap.degree + 1)]
print("G1 = ")
display(tau_G1)

# G2[τ^0], G2[τ^1], ..., G2[τ^{m}]
tau_G2 = [bn128.multiply(bn128.G2, int(tau**i)) for i in range(poly_qap.degree + 1)]
print("G2 = ")
display(tau_G2)

alpha = GF.Random()
alpha_G1 = bn128.multiply(bn128.G1, int(alpha))
print("[alpha]G1")
display(alpha_G1)

beta = GF.Random()
beta_G1 = bn128.multiply(bn128.G1, int(beta))
beta_G2 = bn128.multiply(bn128.G2, int(beta))
print("[beta]G1")
display(beta_G1)
print("[beta]G2")
display(beta_G2)

print("-" * 30)
# end PHASE I setup

# PHASE II setup
print("trusted setup phase two")
print("-" * 30)

constraint_poly_a_vector = constraint_poly_vectors[0]
constraint_poly_b_vector = constraint_poly_vectors[1]
constraint_poly_c_vector = constraint_poly_vectors[2]

# compute poly K on behalf of prover/verifier since prover/verifier does not know alpha and beta
# K_i(\tau) = (C_{i} (\tau) + \beta A_i (\tau) + \alpha B_i (\tau))
k_commitment_vector = []
for i in range(len(constraint_poly_c_vector)):
    cm = bn128.Z1
    cm = bn128.add(cm, commit_poly(constraint_poly_c_vector[i], tau_G1))
    cm = bn128.add(
        cm, bn128.multiply(commit_poly(constraint_poly_a_vector[i], tau_G1), int(beta))
    )
    cm = bn128.add(
        cm,
        bn128.multiply(commit_poly(constraint_poly_b_vector[i], tau_G1), int(alpha)),
    )
    k_commitment_vector.append(cm)

k_commitment_vector_1 = []
for i in range(len(constraint_poly_c_vector)):
    poly = (
        constraint_poly_c_vector[i]
        + beta * constraint_poly_a_vector[i]
        + alpha * constraint_poly_b_vector[i]
    )
    cm = commit_poly(poly, tau_G1)
    k_commitment_vector_1.append(cm)

k_commitment_vector_2 = []
for i in range(len(constraint_poly_c_vector)):
    poly = (
        constraint_poly_c_vector[i]
        + beta * constraint_poly_a_vector[i]
        + alpha * constraint_poly_b_vector[i]
    )
    cm = bn128.multiply(bn128.G1, int(poly(tau)))
    k_commitment_vector_2.append(cm)

assert k_commitment_vector == k_commitment_vector_1
assert k_commitment_vector == k_commitment_vector_2
print(" [K]G1")
display(k_commitment_vector)

# For prover, calculate \tau^k \cdot Z_k(\tau) [G_1]_1
commitment_z = commit_poly(poly_z, tau_G1)
commitment_tau_z_vector = []
for i in range(poly_h.degree + 1):
    cm = bn128.multiply(commitment_z, int(tau**i))
    commitment_tau_z_vector.append(cm)

HZ_G1 = [
    bn128.multiply(bn128.G1, int(tau**i * poly_z(tau)))
    for i in range(poly_h.degree + 1)
]

assert HZ_G1 == commitment_tau_z_vector

print("[tau**i * Z(tau)]G1")
display(commitment_tau_z_vector)
print("-" * 30)
# end of PHASE II setup

trusted setup phase one
------------------------------
toxic waste for tau commitment
G1 = 


[(1, 2),
 (3310343813350012453172073579812399759121079687906176377182825140169073995334,
  17672863612352166519818915341175850089597300240068366169057240681440518840472),
 (6981922066239633482285637555425687931259815013797141751230825192019759632784,
  11727778829703425690795631377810655294629652012230224320663014587276112543120),
 (16561650847457344121550215058576003274908969694392719332913190934011908566752,
  9839328434942511238362542750820588321024424015582665243436998774230227068223),
 (12930333231870885899615323267072627053279465768996811008374150468553173011545,
  6672947933268306567137180340645801669001345861709298074237764922455669486159),
 (9861703507434402262526561581664746967002111403593258321901490144468879779704,
  6320303684482197856778427905333488496649271827169473345795217724840572983923),
 (20534201710578327053980975883539199471695507048495185120116784846388029607680,
  10113278915095273403342506616781165922519609117536408132923126467696199153814)]

G2 = 


[((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634),
  (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531)),
 ((7448883114654723318073946176847807726056536537934070256124415333809912649327, 3645717234639632867422508209816997347640573389502328925603597655819516481335),
  (3978372668672438823368519726533593508234320403947609736874741929362930497610, 5618155499656725307440188847938419018790372685696250427122648285408237712359)),
 ((6851016304294767474706615197796235770703967326718527952638487215537695103003, 20921803241168634616767386906699689346085548620797646500818372295571776511891),
  (9607563680717805384034046318559593804472501485263273167762603571900036110857, 19181254148771554990155028839695688526791984222940051459187923420195078496223)),
 ((736173594879779066922873398293

[alpha]G1


...

------------------------------
trusted setup phase two
------------------------------
 [K]G1


[(16013072081625774486478422251104372492259058609112414148296598570909516007152,
  10504586258872831961426685084594372045575807569561526947638446479784087006246),
 (619782576310242571604499376367801696654254373306766685458104868421291426660,
  10311289634261744140978972911607785984279824658274864073716004649866555726718),
 (5253942016711780316133708778228852840391300249186461526754586631353773216445,
  555334434687879297194764938712095373048815024867617382590319492952766911608),
 (19290171659764672111631952234670916436769458508045521333026396113185467453190,
  4530670199163597009737967538320601302985689521483636831248622512831289640504),
 (97040015648526193123982698947730706637520667040200105070289627538705424628,
  14967267559162315593339788518977233562731559368454557998493777727604564440649),
 (9323810696165403436675008043491938001952254938172360537467703587661076357789,
  2617747047105460965143774608247367288066352380133711252941327766510744841882)]

[tau**i * Z(tau)]G1


[(5610389789189384874297361062900620045098136214017482324614279308887137680268,
  16700735501301959812280412518643161716287832890640043131240834401913633070086),
 (12612536134542115364574480046839646022485974118289680512608028885896101763447,
  13709574124631114877487716111882962132053502847075975876347683013600258306477),
 (18443286757739264553595983722740362857304635798514428524579833960929152718198,
  18862678721798799673631126782257755349134872616404893022145628530082591566542)]

------------------------------


## Prover Part (No Zero Knowledge)

Because we introduce these shifts, we need to modify the equation

$
\underbrace{\left( \alpha + \sum_{j} A_{j} (x) z_j \right)}_{A} \underbrace{\left( \beta + \sum_j B_{j} (x) z_j \right)}_{B} \\
= \alpha \beta + \alpha \sum_j B_{j} (x) z_j + \beta \sum_{j} A_{j} (x) z_j + \underline{\sum_{j} A_{j} (x) z_j \sum_j B_{j} (x) z_j} \\
= \alpha \beta + \alpha \sum_j B_{j} (x) z_j + \beta \sum_{j} A_{j} (x) z_j + {\sum_k C_{k} (x) z_k + h(x)Z_D(x)} \\
= \alpha \beta + \underbrace{\gamma \left( \sum_{i = 0}^{i = k - 1} \gamma^{-1} (C_{i} (x) + \beta A_i (x) + \alpha B_i (x)) z_i \right)}_{\text{public input}} + \underbrace{\delta \left( \sum_{j = k}^{j = n - 1} \delta^{- 1} (C_{j}(x) + \beta A_j(x) + \alpha B_j(x)) z_j \right)}_{\text{private input}} + \delta (\delta^{-1} h(x) Z_D(x)) \\
= \alpha \beta + \underbrace{\gamma \left( \sum_{i = 0}^{i = k - 1} \gamma^{-1} (C_{i} (x) + \beta A_i (x) + \alpha B_i (x)) z_i \right)}_{\text{public input}} + \underbrace{\delta \left( \sum_{j = k}^{j = n - 1} \delta^{- 1} (C_{j}(x) + \beta A_j(x) + \alpha B_j(x)) z_j + \delta^{-1} h(x) Z_D(x)\right)}_{\text{private input}}  \\
$

To ensure that the proof is zero-knowledge, the prover sample two random scalars, $r$ and $s$ to prevent verifier from guessing witness values.

The prover can compute the three elements of the proof, $\pi = ([\pi_1 ]_1 , [\pi_2 ]_2 , [\pi_3 ]_1)$ by doing the following calculations:

$
[\pi_1 ]_1 = [\alpha]_1 + \sum z_k [A_k(\tau)]_1 \\
[\pi_2 ]_2 = [\beta]_2 + \sum z_k [B_k(\tau)]_2 \\
[\pi_2 ]_1 = [\beta]_1 + \sum z_k [B_k(\tau)]_1  \\
[\delta^{-1} \cdot h(t)z(\tau)]_1 = \delta^{-1} \cdot \sum h_i [\tau^{i}Z_i (\tau)]_1 \\
[\pi_3 ]_1 = \sum w_i [K_i^p ]_1 + [\delta^{-1}h(\tau)z(\tau)]_1
$

In [7]:
# Prover generate proof
print("\nProver generate proof\n")
print("-" * 30)
gamma = GF.Random()
delta = GF.Random()

print("\ngenerate gamma and delta to differentiate public and private witness\n")

delta_G1 = bn128.multiply(bn128.G1, int(delta))
delta_G2 = bn128.multiply(bn128.G2, int(delta))


# pi1_G1 = (alpha + A(x)) [G_1]_1
pi1_G1 = alpha_G1
A_witness_commitment_G1 = commit_poly(A_witness_poly, tau_G1)
pi1_G1 = bn128.add(pi1_G1, A_witness_commitment_G1)

# pi2_G2 = (beta + B(x)) [G_2]_2
pi2_G2 = beta_G2
B_witness_commitment_G2 = commit_poly(B_witness_poly, tau_G2)
pi2_G2 = bn128.add(pi2_G2, B_witness_commitment_G2)

ell = 2  # number of public input in witness vector, {1, out}


# pi3_G1 = delta^-1 * (private k_commitment * z_i  + h(tau)z(tau))

C2_G1 = bn128.Z1
# K_v * witness
for i in range(ell, len(k_commitment_vector)):
    entry = bn128.multiply(k_commitment_vector[i], int(w[i]))
    C2_G1 = bn128.add(C2_G1, entry)

C2_G1 = bn128.multiply(C2_G1, int(delta**-1))

# \delta^{-1} h(tau)Z(tau)[G_1]_1
hz_G1 = commit_poly(poly_h, commitment_tau_z_vector)
hz_G1 = bn128.multiply(hz_G1, int(delta**-1))

pi3_G1 = bn128.add(C2_G1, hz_G1)

print("[pi1]G1 = ", pi1_G1)
print("[pi2]G2 = ", pi2_G2)
print("[pi3]G1 = ", pi3_G1)

print("[delta]G1 = ", delta_G1)
print("[delta]G2 = ", delta_G2)


Prover generate proof

------------------------------

generate gamma and delta to differentiate public and private witness

[pi1]G1 =  (10718595111368112521941806878916262421202585780472388992125838874357356608945, 363374329349765577935555970266930996896401143371839896591740180036631734683)
[pi2]G2 =  ((20588936413971171589143658175627920544663041718868269382857568754006364419347, 7087614294939287886386872230450307984126485028177904335147619027936498192400), (17578763278547485903077312475638831386831876641591214528595214802464546599079, 19013408912221019298667073993517416034544594269939877148954592411470208463593))
[pi3]G1 =  (17222019539406291029296543390180257138666061813012071498311710363572822004985, 3686666225864009517165691898701586204110409635872985670895597077117429229126)
[delta]G1 =  (5819078839081810986254726647090530036068392631363692034482460911902998394226, 8080966965814537226332416263223742691030723102976522483693832257512345530293)
[delta]G2 =  ((803143679988035350237

## Verifier Part (No Zero Knowledge)

$
[\pi_1]_1 \dagger [\pi_2]_2 \stackrel{?}{=} [\alpha]_1 \dagger [\beta]_2 * [C_1]_1 \dagger [\gamma]_2 * [\pi_3]_1 \dagger [\delta]_2
$

In [8]:
import py_ecc.bn128 as bn128
import py_ecc.bn128.bn128_pairing as pairing

print("\nProof Verification")
print("-" * 20)

C1_G1 = bn128.Z1
for i in range(ell):
    entry = bn128.multiply(k_commitment_vector[i], int(w[i]))
    C1_G1 = bn128.add(C1_G1, entry)
C1_G1 = bn128.multiply(C1_G1, int(gamma**-1))

gamma_G2 = bn128.multiply(bn128.G2, int(gamma))

assert pairing.pairing(pi2_G2, pi1_G1) == (
    pairing.pairing(beta_G2, alpha_G1)
    * pairing.pairing(gamma_G2, C1_G1)
    * pairing.pairing(delta_G2, pi3_G1)
), "Pairing check failed"


Proof Verification
--------------------


## Prover Part (With Zero Knowledge)

To ensure that the proof is zero-knowledge, the prover sample two random scalars, $r$ and $s$.

The prover can compute the three elements of the proof, $\pi = ([\pi_1 ]_1 , [\pi_2 ]_2 , [\pi_3 ]_1)$ by doing the following calculations:

$
[\pi_1 ]_1 = [\alpha]_1 + \sum z_k [A_k(\tau)]_1 + r [\delta]_1 \\
[\pi_2 ]_2 = [\beta]_2 + \sum z_k [B_k(\tau)]_2 + s[\delta]_2 \\
[\pi_2 ]_1 = [\beta]_1 + \sum z_k [B_k(\tau)]_1 + s[\delta]_1 \\
[\delta^{-1} \cdot h(t)z(\tau)]_1 = \delta^{-1} \cdot \sum h_i [\tau^{i}Z_i (\tau)]_1 \\
[\pi_3 ]_1 = \sum w_i [K_i^p ]_1 + [\delta^{-1}h(\tau)z(\tau)]_1 + s[\pi_1 ]_1 + r [\pi_2 ]_1 - rs [\delta]_1
$

In [9]:
# Prover generate proof
print("\nProver generate proof\n")
print("-" * 30)
gamma = GF.Random()
delta = GF.Random()
r = GF.Random()
s = GF.Random()

# r = random.getrandbits(256)
# r = r % bn128.curve_order

print("\ngenerate gamma and delta to differentiate public and private witness\n")

delta_G1 = bn128.multiply(bn128.G1, int(delta))
delta_G2 = bn128.multiply(bn128.G2, int(delta))

assert bn128.multiply(bn128.G1, int(r * delta)) == bn128.multiply(delta_G1, int(r))
assert bn128.multiply(bn128.neg(bn128.G1), int(r * s * delta)) == bn128.multiply(
    delta_G1, int(-r * s)
)

# pi1_G1 = (alpha + A(x) + r \delta) [G_1]_1
pi1_G1 = alpha_G1
A_witness_commitment_G1 = commit_poly(A_witness_poly, tau_G1)
pi1_G1 = bn128.add(pi1_G1, A_witness_commitment_G1)
pi1_G1 = bn128.add(pi1_G1, bn128.multiply(delta_G1, int(r)))
print(f"[{chr(960)}1]G1: ", pi1_G1)

# pi2_G1 = (beta + B(x) + s \delta) [G_1]_1
pi2_G1 = beta_G1
B_witness_commitment_G1 = commit_poly(B_witness_poly, tau_G1)
pi2_G1 = bn128.add(pi2_G1, B_witness_commitment_G1)
pi2_G1 = bn128.add(pi2_G1, bn128.multiply(delta_G1, int(s)))

# pi2_G2 = (beta + B(x) + s \delta) [G_2]_2
pi2_G2 = beta_G2
B_witness_commitment_G2 = commit_poly(B_witness_poly, tau_G2)
pi2_G2 = bn128.add(pi2_G2, B_witness_commitment_G2)
pi2_G2 = bn128.add(pi2_G2, bn128.multiply(delta_G2, int(s)))

k = 2  # number of public input in witness vector, {1, out}

# pi3_G1 = private k_commitment * z_i * delta^-1 + h(tau)z(tau) + delta(s * pi1 + r * pi2 - r*s*delta)

private_C_G1 = bn128.Z1
# K_v * witness
for i in range(k, len(k_commitment_vector)):
    entry = bn128.multiply(k_commitment_vector[i], int(w[i]))
    private_C_G1 = bn128.add(private_C_G1, entry)

private_C_G1 = bn128.multiply(private_C_G1, int(delta**-1))

# \delta^{-1} h(tau)Z(tau)[G_1]_1
hz_G1 = commit_poly(poly_h, commitment_tau_z_vector)
hz_G1 = bn128.multiply(hz_G1, int(delta**-1))

cross_G1 = bn128.Z1
cross_G1 = bn128.add(cross_G1, bn128.multiply(pi1_G1, int(s)))
cross_G1 = bn128.add(cross_G1, bn128.multiply(pi2_G1, int(r)))
cross_G1 = bn128.add(cross_G1, bn128.multiply(delta_G1, int(-r * s)))

pi3_G1 = bn128.add(private_C_G1, hz_G1)
pi3_G1 = bn128.add(pi3_G1, cross_G1)

print("[pi1]G1 = ", pi1_G1)
print("[pi2]G2 = ", pi2_G2)
print("[pi3]G1 = ", pi3_G1)

print("[delta]G1 = ", delta_G1)
print("[delta]G2 = ", delta_G2)


Prover generate proof

------------------------------

generate gamma and delta to differentiate public and private witness

[π1]G1:  (15684943727443821256117791169340323818339980784270053105095118833365005412327, 10679997383457523051382868102129192021840584142313769888512456927250074243398)
[pi1]G1 =  (15684943727443821256117791169340323818339980784270053105095118833365005412327, 10679997383457523051382868102129192021840584142313769888512456927250074243398)
[pi2]G2 =  ((15976323390082003237185140221851220295395261234139993787468341012897937073698, 6190576918801488629916638272648464140116987641560229042143635483569534554729), (21769405015677950937942572132185609855886621743636928318835413876693851020964, 8277493466588703105941139510309914099486039602912975180704576296457524146672))
[pi3]G1 =  (6111768291579157929610418207914469442012826078978922547643346854848061164089, 18042066836860712557049150602138142518885603716478273628035652497937804185579)
[delta]G1 =  (21635396538531816202260

## Verifier Part

$
[\pi_1]_1 \dagger [\pi_2]_2 \stackrel{?}{=} [\alpha]_1 \dagger [\beta]_2 * [C_1]_1 \dagger [\gamma]_2 * [\pi_3]_1 \dagger [\delta]_2
$

In [10]:
import py_ecc.bn128 as bn128
import py_ecc.bn128.bn128_pairing as pairing

print("\nProof Verification")
print("-" * 20)

public_C_G1 = bn128.Z1
for i in range(k):
    entry = bn128.multiply(k_commitment_vector[i], int(w[i]))
    public_C_G1 = bn128.add(public_C_G1, entry)
public_C_G1 = bn128.multiply(public_C_G1, int(gamma**-1))

gamma_G2 = bn128.multiply(bn128.G2, int(gamma))

assert pairing.pairing(pi2_G2, pi1_G1) == (
    pairing.pairing(beta_G2, alpha_G1)
    * pairing.pairing(gamma_G2, public_C_G1)
    * pairing.pairing(delta_G2, pi3_G1)
), "Pairing check failed"


Proof Verification
--------------------
